In [18]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [20]:
url = 'https://www.jiomart.com/c/groceries/2?prod_mart_groceries_products_popularity%5Bpage%5D=2'

In [22]:
response = requests.get(url)# Run the async function

In [23]:
response.status_code

200

In [24]:
soup = BeautifulSoup(response.content,'html.parser')

In [26]:
results = soup.find_all('div',{'class':'plp-card-details-container'})

In [27]:
len(results)

12

In [28]:
#title
results[0].find('div',{'class':'plp-card-details-name line-clamp jm-body-xs jm-fc-primary-grey-80'}).get_text().strip()

'Super Sarvottam Physically Refined Rice Bran Oil 1 L'

In [29]:
# price
results[0].find('span',{'class':'jm-heading-xxs jm-mb-xxs'}).get_text().strip()

'₹97.00'

In [30]:
#discount
results[0].find('span',{'class':'jm-badge'}).get_text().strip()

'51% OFF'

In [31]:
# Puting Everything in loop

product_name = []
product_price = []
product_discount = []
for i in range(len(results)):
    badge_element = results[i].find('span', {'class': 'jm-badge'})
    product_name.append(results[i].find('div',{'class':'plp-card-details-name line-clamp jm-body-xs jm-fc-primary-grey-80'}).get_text().strip())
    product_price.append(results[i].find('span',{'class':'jm-heading-xxs jm-mb-xxs'}).get_text().strip())
    if badge_element is not None:
        text_inside_badge = badge_element.get_text().strip()
        if text_inside_badge:  # Check if the text is not an empty string
            product_discount.append(text_inside_badge)
        else:
            # Handle the case when the text is empty, e.g., set a default value
            product_discount.append("None")  # Replace "No Discount" with your desired default value
    else:
        # Handle the case when the element is not found, e.g., set a default value
        product_discount.append("None")

In [32]:
product_name

['Super Sarvottam Physically Refined Rice Bran Oil 1 L',
 'Priya Fortified With Vitamin A & D Refined Sunflower Oil 1 L',
 'Surf Excel Easy Wash Detergent Powder 5 kg',
 'Closeup Everfresh+ Red Hot Gel Toothpaste 150 g (Pack of 2)',
 'Bharat Chana Dal 1 Kg',
 'Brooke Bond Red Label Strong Blend Tea 1 kg',
 'Maaza Mango Drink 1.2 L',
 'Pears Pure & Gentle Soap with Natural Oils 125 g (Buy 4 Get 1 Free)',
 'Surf Excel Detergent Bar 200 g (Pack of 4)',
 'Rin Detergent Bar 110 g',
 'Kissan Fresh Tomato Ketchup 1.1 kg',
 'Tide Jasmine & Rose Detergent Powder 5 kg (Get Extra 1 kg Free)']

In [33]:
product_price

['₹97.00',
 '₹96.00',
 '₹500.00',
 '₹119.00',
 '₹60.00',
 '₹450.00',
 '₹50.00',
 '₹243.00',
 '₹96.00',
 '₹9.00',
 '₹98.00',
 '₹715.00']

In [34]:
product_discount

['51% OFF',
 '40% OFF',
 '31% OFF',
 '48% OFF',
 'None',
 '25% OFF',
 '33% OFF',
 '35% OFF',
 '21% OFF',
 '10% OFF',
 '34% OFF',
 '7% OFF']

In [35]:
data = {'Product Name':product_name,'Price':product_price,'Discount':product_discount}
df = pd.DataFrame(data)


In [36]:
df['Discount'] = df['Discount'].replace('None','0% OFF')

In [37]:
df

,Product Name,Price,Discount
0,Super Sarvottam Physically Refined Rice Bran O...,₹97.00,51% OFF
1,Priya Fortified With Vitamin A & D Refined Sun...,₹96.00,40% OFF
2,Surf Excel Easy Wash Detergent Powder 5 kg,₹500.00,31% OFF
3,Closeup Everfresh+ Red Hot Gel Toothpaste 150 ...,₹119.00,48% OFF
4,Bharat Chana Dal 1 Kg,₹60.00,0% OFF
5,Brooke Bond Red Label Strong Blend Tea 1 kg,₹450.00,25% OFF
6,Maaza Mango Drink 1.2 L,₹50.00,33% OFF
7,Pears Pure & Gentle Soap with Natural Oils 125...,₹243.00,35% OFF
8,Surf Excel Detergent Bar 200 g (Pack of 4),₹96.00,21% OFF
9,Rin Detergent Bar 110 g,₹9.00,10% OFF


In [38]:
# df['Original Price'] = df['Price']+ (df['Price']*0.5)
df['Price'] = df['Price'].str[1:]

In [39]:
df['Price']=df['Price'].astype(float)

In [40]:
df['Discount'] = df['Discount'].str[0:-5]

In [41]:
df['Discount'] = df['Discount'].astype(float)

In [42]:
df['Original Price'] = df['Price'] / (1 - df['Discount']/100)

In [43]:
df['Original Price'] = round(df['Original Price'])

In [44]:
df

,Product Name,Price,Discount,Original Price
0,Super Sarvottam Physically Refined Rice Bran O...,97.0,51.0,198.0
1,Priya Fortified With Vitamin A & D Refined Sun...,96.0,40.0,160.0
2,Surf Excel Easy Wash Detergent Powder 5 kg,500.0,31.0,725.0
3,Closeup Everfresh+ Red Hot Gel Toothpaste 150 ...,119.0,48.0,229.0
4,Bharat Chana Dal 1 Kg,60.0,0.0,60.0
5,Brooke Bond Red Label Strong Blend Tea 1 kg,450.0,25.0,600.0
6,Maaza Mango Drink 1.2 L,50.0,33.0,75.0
7,Pears Pure & Gentle Soap with Natural Oils 125...,243.0,35.0,374.0
8,Surf Excel Detergent Bar 200 g (Pack of 4),96.0,21.0,122.0
9,Rin Detergent Bar 110 g,9.0,10.0,10.0


In [45]:
df.to_csv('Jio_Mart.csv')